In [1]:
import io
import os
import re
import sys
import csv
import gzip
import time
import json
import pickle
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
from scipy.stats import pearsonr
# import sqlite3
# from PIL import Image
import matplotlib.pyplot as plt
# from plotnine import *
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
# from collections import Counter
# import seaborn as sns
# import matplotlib.colors as mcolors
# from matplotlib.ticker import FuncFormatter
import itertools
import collections
import pyreadr
import psa
import pywfa
from IPython.display import display_html, IFrame
# import patchworklib as pw
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *
from warnings import simplefilter
from hla_phase import *

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None

# Genuine logics changed from the main script were marked by string **X** in the original R files.
# Simplicity changes were marked by string **Y**

<Figure size 100x100 with 0 Axes>

In [2]:
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')

ref_samples = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/oneKG.samples', sep = ' ')
hlatypes = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/20181129_HLA_types_full_1000_Genomes_Project_panel.txt', sep = '\t')
ref_samples_removed = ref_samples[~ref_samples['SAMPLE'].isin(hlatypes['Sample ID'].tolist())]
samples_to_remove = ref_samples_removed['SAMPLE'].tolist()
hlatypes = hlatypes[~hlatypes['Sample ID'].isin(samples_to_remove)].sort_values(by = 'Sample ID').reset_index(drop = True)

phased_vcf = "/well/band/users/rbx225/recyclable_files/ref_panels/oneKG/oneKG.chr6.vcf.gz"
reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))

return_dict = phase_hla_on_haplotypes(gene = gene, 
                            ipd_gen_file_dir = ipd_gen_file_dir, 
                            hla_gene_information = hla_gene_information,
                            hlatypes = hlatypes,
                            phased_vcf = phased_vcf, 
                            reference_allele_ary = reference_allele_ary, 
                            read_from_QUILT = False, 
                            subset_vcf_samples = None,
                            sample_linker = None)
hlatypes = return_dict['hlatypes']

individual = 'NA12878'
ix = hlatypes.index[hlatypes['Sample ID'] == 'NA12878'][0]
display_indices = np.arange(10)
html = '1KG-DRB1.html'

res = visualise_phase(gene, ix, hlatypes, return_dict, both_het = True)
compare_phase(display_indices, res, save_html = True, save_name = html)

plot_first_step_phase(return_dict)
plot_last_step_phase(return_dict)
# reference_exclude_samplelist_file = ''
# if reference_exclude_samplelist_file != '':
#     samples_to_remove = samples_to_remove + lcwgsus.read_tsv_as_lst(reference_exclude_samplelist_file)
# if samples_to_remove is not none, save to "hlauntyped.exclude.txt"

IndexError: positional indexers are out-of-bounds

In [16]:
quilt_phase = pd.read_csv('/well/band/users/rbx225/recyclable_files/hla/1KG_phased_' + gene + '.tsv', sep = '\t')

phase_df = return_dict['phase_df']
phased_indices = phase_df.index[(phase_df['final_step_phase1'] | phase_df['final_step_phase2'])]
print('Number of unphased samples:', len(phase_df) - len(phased_indices))

df1 = phase_df.loc[phased_indices,:]
df2 = quilt_phase.loc[phased_indices,:]

matched_haps = df1['final_step_phase1'] == df2['phase1_final']
percet_matched_haps = (matched_haps.sum() / len(df1)) * 100
print(f'Percentage agreed with QUILT phasing for {gene}: ', percet_matched_haps, '%')

Number of unphased samples: 3
Percentage agreed with QUILT phasing for DRB1:  97.38791423001949 %


In [ ]:
# For GAMCC
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
samples_to_remove = []

gamcc_hla = lcwgsus.read_hla_direct_sequencing(retain = 'fv', unique_two_field = False)
gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Two field1', 'Two field2']].reset_index(drop = True)

colnames = ['Sample ID'] + [label for g in HLA_GENES for label in [f'HLA-{g} 1', f'HLA-{g} 2']]
hlatypes = pd.DataFrame(columns = colnames)
for s in gamcc_hla['SampleID'].unique():
    tmp = gamcc_hla[gamcc_hla['SampleID'] == s]
    row = [s] + tmp[['Two field1', 'Two field2']].values.ravel().tolist()
    hlatypes.loc[len(hlatypes)] = row

reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))
phased_vcf = "/well/band/users/rbx225/GAMCC/results/imputation/vcfs/malariaGen_v1_b38/quilt.chr6.vcf.gz"

subset_vcf_samples = lcwgsus.read_tsv_as_lst('/well/band/users/rbx225/GAMCC/data/sample_tsvs/fv_gm_names.tsv')
subset_vcf_samples = ','.join(subset_vcf_samples)

sample_linker = pd.read_csv('/well/band/users/rbx225/GAMCC/' + SAMPLE_LINKER_FILE)
sample_linker = {k:v for k, v in zip(sample_linker['Sample_Name'], sample_linker['Chip_Name'])}

return_dict = phase_hla_on_haplotypes(gene = gene, 
                            ipd_gen_file_dir = ipd_gen_file_dir, 
                            hla_gene_information = hla_gene_information, 
                            hlatypes = hlatypes,
                            phased_vcf = phased_vcf, 
                            reference_allele_ary = reference_allele_ary, 
                            read_from_QUILT = True, 
                            subset_vcf_samples = subset_vcf_samples,
                            sample_linker = sample_linker)

hlatypes = return_dict['hlatypes']

In [23]:
unphased_samples_to_remove = hlatypes.loc[indices, 'Sample ID'].tolist()
samples_to_remove = samples_to_remove + unphased_samples_to_remove

In [20]:
# QUILT phasing is in the order of paternal-maternal
retained_samples = hlatypes['Sample ID'].tolist()
trios = pd.read_csv('/well/band/users/rbx225/recyclable_files/ref_panels/oneKG/integrated_call_samples_v3.20200731.ALL.ped', sep = '\t')
trios = trios[trios['Individual ID'].isin(retained_samples)]
trios = trios[(trios['Paternal ID'].isin(retained_samples)) | (trios['Maternal ID'].isin(retained_samples))]
trios = trios[['Individual ID', 'Paternal ID', 'Maternal ID']]

trios['allele1'] = 'N/A'
trios['allele2'] = 'N/A'
undetermined_trios = []

trios = trios.apply(func = phase_trios, args = (gene, hlatypes), axis = 1)

kgp_samples_idx = hlatypes.index[hlatypes['Sample ID'].isin(trios['Individual ID'])].to_numpy()
kgp_samples_phase = phased1[kgp_samples_idx]

phased_res = pd.DataFrame({'Sample ID': trios['Individual ID'].tolist(), 'allele1': reftypes1[kgp_samples_idx], 'allele2': reftypes2[kgp_samples_idx]})

for i in range(len(phased_res)):
    if kgp_samples_phase[i] == False:
        tmp = phased_res.iloc[i, 1]
        phased_res.iloc[i, 1] = phased_res.iloc[i, 2]
        phased_res.iloc[i, 2] = tmp

res = pd.merge(phased_res, trios[['Individual ID', 'allele1', 'allele2']], left_on = 'Sample ID', right_on = 'Individual ID', suffixes = ('_pred', '_true'))
res = res.drop(columns = 'Individual ID')
res

NameError: name 'phase_trios' is not defined

In [79]:
full_start = return_dict['b38_start']
full_end = return_dict['b38_end']
hlatypes = return_dict['hlatypes']

# full_start = 32568456
# full_end = 32589742

retained_samples = hlatypes['Sample ID'].tolist()
trios = pd.read_csv('/well/band/users/rbx225/recyclable_files/ref_panels/oneKG/integrated_call_samples_v3.20200731.ALL.ped', sep = '\t')
trios = trios[trios['Individual ID'].isin(retained_samples)]
trios = trios[(trios['Paternal ID'].isin(retained_samples)) & (trios['Maternal ID'].isin(retained_samples))]
trios = trios[['Individual ID', 'Paternal ID', 'Maternal ID']]

trio_samples_all = trios.values.ravel().tolist()

df = read_vcf(
        full_start - 5000, 
        full_end + 5000, 
        phased_vcf, 
        hlatypes, 
        False, 
        ','.join(trio_samples_all), 
        None)

df = df[(df['ref'].str.len() == 1) & (df['alt'].str.len() == 1)]

df_up = df[df['pos'] < full_start]
df_down = df[df['pos'] > full_end]

phase_df = return_dict['phase_df']
phase_df = phase_df[phase_df['Sample'].isin(trio_samples_all)]

trio_haps = {}

for child in trios['Individual ID']:
    mat_id, pat_id = trios[trios['Individual ID'] == child][['Maternal ID', 'Paternal ID']].values[0]
    
    child_alleles = phase_df[phase_df['Sample'] == child][['allele1', 'allele2']].values[0]
    mat_alleles = phase_df[phase_df['Sample'] == mat_id][['allele1', 'allele2']].values[0]
    pat_alleles = phase_df[phase_df['Sample'] == pat_id][['allele1', 'allele2']].values[0]
    
    df_up_child = df_up[['pos', 'ref', 'alt'] + [child, mat_id, pat_id]]
    df_up_child = df_up_child[~df_up_child[child].str.contains(r'\.')]
    df_up_child['pat'] = df_up_child[child].str.split('|').str.get(0)
    df_up_child['mat'] = df_up_child[child].str.split('|').str.get(1)
    het_sites = (df_up_child['mat'] != df_up_child['pat'])
    
    upstream_idx = het_sites[het_sites].index[-1]
    ref, alt, mat, pat = df_up_child.loc[upstream_idx, ['ref', 'alt', 'mat', 'pat']]
    
    mat_genotypes = df_up_child.loc[upstream_idx, mat_id].split('|')
    pat_genotypes = df_up_child.loc[upstream_idx, pat_id].split('|')
    
    if mat_genotypes[0] == '0':
        mat_hap1 = ref + '-' + mat_alleles[0] + '-'
    elif mat_genotypes[0] == '1':
        mat_hap1 = alt + '-' + mat_alleles[0] + '-'
    else:
        mat_hap1 = 'N' + '-' + mat_alleles[0] + '-'
        
    if mat_genotypes[1] == '0':
        mat_hap2 = ref + '-' + mat_alleles[1] + '-'
    elif mat_genotypes[1] == '1':
        mat_hap2 = alt + '-' + mat_alleles[1] + '-'
    else:
        mat_hap2 = 'N' + '-' + mat_alleles[1] + '-'
    
    if pat_genotypes[0] == '0':
        pat_hap1 = ref + '-' + pat_alleles[0] + '-'
    elif pat_genotypes[0] == '1':
        pat_hap1 = alt + '-' + pat_alleles[0] + '-'
    else:
        pat_hap1 = 'N' + '-' + pat_alleles[0] + '-'
        
    if pat_genotypes[1] == '0':
        pat_hap2 = ref + '-' + pat_alleles[1] + '-'
    elif pat_genotypes[1] == '1':
        pat_hap2 = alt + '-' + pat_alleles[1] + '-'
    else:
        pat_hap2 = 'N' + '-' + pat_alleles[1] + '-'
    
    if pat == '0':
        child_hap1 = ref + '-' + child_alleles[0] + '-'
        child_hap2 = alt + '-' + child_alleles[1] + '-'
    elif pat == '1':
        child_hap1 = alt + '-' + child_alleles[0] + '-'
        child_hap2 = ref + '-' + child_alleles[1] + '-'
    else:
        child_hap1 = 'N' + '-' + child_alleles[0] + '-'
        child_hap2 = 'N' + '-' + child_alleles[1] + '-'
    
    df_down_child = df_down[['pos', 'ref', 'alt'] + [child, mat_id, pat_id]]
    df_down_child = df_down_child[~df_down_child[child].str.contains(r'\.')]
    df_down_child['pat'] = df_down_child[child].str.split('|').str.get(0)
    df_down_child['mat'] = df_down_child[child].str.split('|').str.get(1)
    het_sites = (df_down_child['mat'] != df_down_child['pat'])
    downstream_idx = het_sites[het_sites].index[0]
    ref, alt, mat, pat = df_down_child.loc[downstream_idx, ['ref', 'alt', 'mat', 'pat']]
    
    mat_genotypes = df_down_child.loc[downstream_idx, mat_id].split('|')
    pat_genotypes = df_down_child.loc[downstream_idx, pat_id].split('|')
    
    if mat_genotypes[0] == '0':
        mat_hap1 = mat_hap1 + ref
    elif mat_genotypes[0] == '1':
        mat_hap1 = mat_hap1 + alt
    else:
        mat_hap1 = mat_hap1 + 'N'
        
    if mat_genotypes[1] == '0':
        mat_hap2 = mat_hap2 + ref
    elif mat_genotypes[1] == '1':
        mat_hap2 = mat_hap2 + alt
    else:
        mat_hap2 = mat_hap2 + 'N'
    
    if pat_genotypes[0] == '0':
        pat_hap1 = pat_hap1 + ref
    elif pat_genotypes[0] == '1':
        pat_hap1 = pat_hap1 + alt
    else:
        pat_hap1 = pat_hap1 + 'N'
        
    if pat_genotypes[1] == '0':
        pat_hap2 = pat_hap2 + ref
    elif pat_genotypes[1] == '1':
        pat_hap2 = pat_hap2 + alt
    else:
        pat_hap2 = pat_hap2 + 'N'
    
    if pat == '0':
        child_hap1 = child_hap1 + ref
        child_hap2 = child_hap2 + alt
    elif pat == '1':
        child_hap1 = child_hap1 + alt
        child_hap2 = child_hap2 + ref
    else:
        break
    
    trio_haps[child] = [child_hap1, child_hap2, pat_hap1, pat_hap2, mat_hap1, mat_hap2]
    
for v in trio_haps.values():
    print(v)

['G-14:54-T', 'T-09:01-G', 'G-08:03-T', 'G-14:54-G', 'G-14:04-T', 'T-09:01-G']
['G-11:04-T', 'A-03:01-G', 'G-11:04-T', 'A-12:01-G', 'A-03:01-G', 'A-14:54-T']
['C-12:02-C', 'T-12:02-A', 'C-12:02-C', 'T-10:01-C', 'C-12:02-C', 'T-08:03-A']
['G-03:01-T', 'T-01:01-A', 'G-03:01-T', 'G-15:01-A', 'T-01:01-A', 'T-01:01-A']
['G-12:01-T', 'A-16:02-A', 'A-13:01-T', 'G-12:01-T', 'A-16:02-A', 'G-11:01-T']
['C-07:01-G', 'G-03:01-A', 'C-07:01-G', 'G-15:01-A', 'G-16:02-A', 'G-03:01-A']
['T-01:02-T', 'G-14:06-G', 'G-01:02-T', 'T-04:11-T', 'G-04:07-T', 'G-14:06-G']


In [ ]:
# RETAIN THIS!
# gamcc_hla = pd.read_csv('/well/band/users/rbx225/GAMCC/data/hla_direct_sequencing/HLA_direct_sequencing_all.csv')
# gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Included Alleles']]
# for s in gamcc_hla['SampleID'].unique():
#     tmps = gamcc_hla[gamcc_hla['SampleID'] == s]
#     for l in HLA_GENES:
#         tmpl = tmps[tmps['Locus'] == l]
#         repeat = 2 - tmpl.shape[0]
#         if repeat == 2:
#             gamcc_hla.loc[len(gamcc_hla)] = [s, l, np.nan]
#             gamcc_hla.loc[len(gamcc_hla)] = [s, l, np.nan]
#         if repeat == 1:
#             gamcc_hla.loc[len(gamcc_hla)] = [s, l, tmpl.iloc[0,2]]
                
# gamcc_hla = gamcc_hla[gamcc_hla['Locus'] == gene]
# gamcc_hla = gamcc_hla[['SampleID', 'Included Alleles']]

# gamcc_hla = pd.merge(gamcc_hla[::2], gamcc_hla[1::2], on = 'SampleID')
# gamcc_hla.columns = ['Sample ID', 'HLA-DRB1 1', 'HLA-DRB1 2']
# gamcc_hla

# x = gamcc_hla.iloc[0, 1]
# indices = []
# db_alleles = db.index.to_numpy()

# for a in x.split('/'):
#     old_a = a
#     while (gene + '*' + a) not in db.index and ':' in a:
#         a = a.rsplit(':', 1)[0]
#     if ':' in a:
#         indices.append(np.where(db_alleles == gene + '*' + a)[0][0])

# x = db.iloc[np.array(indices), :]

In [78]:
df_up_child

,pos,ref,alt,NA19685,NA19660,NA19661,mat,pat
0,32566587,A,T,0|0,0|0,0|0,0,0
1,32566604,C,G,0|0,0|0,0|0,0,0
2,32566652,T,C,0|0,0|0,0|0,0,0
3,32566682,G,A,1|0,0|0,0|1,1,0
4,32566687,C,T,1|0,0|0,0|1,1,0
5,32566689,T,C,0|0,0|0,0|0,0,0
6,32566695,A,G,0|0,0|0,0|0,0,0
7,32566705,C,T,1|0,0|0,0|1,1,0
8,32566707,G,A,1|0,0|0,0|1,1,0
9,32566720,C,T,0|0,0|0,0|0,0,0
